In [ ]:
from analysis import Bittrex

In [ ]:
# instance olusturuyoruz
analysis = Bittrex()

# kullanılabilecek metodlar: 
#     .get_summary('eth')
#     .tick('eth')
#     .get_market()
#     .get_orderbook('eth')
#     .orderbook_analysis('eth', sample=10)
#     .plot('eth', loop=0, sleep=5)

In [ ]:
# ilgili coin icin request anindaki son degerleri getirir
analysis.get_summary('eth')

In [ ]:
# coin icin son - alis/satis talimatinin deger bilgilerini getirir
# not : get_summary metodundaki verilerle aynidir
ab = ['dgb', 'snt', 'vrm', 'amp', 'pink']
[{ a: analysis.tick(a)}  for a in ab]

In [ ]:
# Bittrex uzerindeki tum coinlerin market bilgilerini getirir
analysis.get_market()

In [ ]:
# verilen coin icin o an acik olan tum alis/satis talimalarini gosterir. 
# Quantity: coinden kac adet oldugu, Rate: BTC cinsinden talimat degeri
analysis.get_orderbook('eth')

In [ ]:
# orderbook uzerinden bir deger analizi cikarilir. sample sayisi kadar son orderlar, 
# bunlarin toplami ve sample sayisi kadar orderin grafik egri degeri hesaplanir
analysis.orderbook_analysis('eth', sample=10)

In [ ]:
# coin icin anlik alis-satis talimatlari ve deger bilgileri grafigi yazdirilir. 
# loop = 0 : surekli ekrana basar, 
# loop = N : N saniye kadar calisir
# sleep = N : her bir request icin N saniye kadar bekler
analysis.plot('dgb', loop=100, sleep=3)

In [ ]:
analysis.plot_file('mtl', path='data/')

In [127]:
#!/usr/bin/env python3.6

import requests, json, time, hmac, hashlib
from matplotlib import pyplot as plt
import multiprocessing
import os
from pandas import read_csv
plt.rcParams['figure.figsize'] = (16,4)
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import math
import bokeh.palettes as palettes
output_notebook()

class Bittrex(object):
    def __init__(self, auth_info=None):
        self.key    = auth_info and auth_info['key']
        self.secret = auth_info and auth_info['secret']
        self.uri    = 'https://bittrex.com/api/v1.1'

    def get_request(self, url, sign):
        return requests.get(url, headers={'apisign': sign}).json()
    
    def infinity(self): 
        while True: 
            yield
    
    def scale(self, data, minX, maxX):
        scaler = MinMaxScaler(feature_range=(minX, maxX), copy=False)
        scaler.fit(data.reshape(-1, 1))
        return scaler.transform(data.reshape(-1, 1)).reshape(-1)
    
    def dispatch(self, output, result_only=True, index=None):
        if index:
            return output.json()['result'][index] if result_only else output.json()
        else:
            return output.json()['result'] if result_only else output.json()
        
    
    def get_balance(self, all=False, result_only=True):
        url    = self.uri + '/account/getbalances?apikey={}&nonce={}'.format(self.key, time.time())
        sign   = hmac.new(self.secret.encode(), url.encode(), hashlib.sha512).hexdigest()
        result = self.get_request(url, sign)
        
        if result_only:
            return result['result'] if all else list(filter(lambda x: x['Balance'] > 0, result['result']))
        else:
            return result if all else {'message': result['message'], 
                                   'success': result['success'], 
                                   'result': list(filter(lambda x: x['Balance'] > 0, result['result']))}
    
    def get_summary(self, c='vrm', base='btc', result_only=True):
        return self.dispatch(
            requests.get(self.uri + '/public/getmarketsummary?market={}-{}'.format(base, c)), 
            result_only=result_only, index=0)
        
    
    def tick(self, c='vrm', base='btc', result_only=True):
        return self.dispatch(
            requests.get(self.uri + '/public/getticker?market={}-{}'.format(base, c)),
            result_only=result_only)

    def get_coins(self, base='BTC'):
        return list(map(lambda x: x['MarketCurrency'], filter(lambda y: 
                           y['BaseCurrency'] == base.upper(), 
                           requests.get(self.uri + '/public/getmarkets/').json()['result'])))
    
    
    def get_market(self, result_only=True):
        return self.dispatch(
            requests.get(self.uri + '/public/getmarkets/'),
            result_only)
    
    def get_orderbook(self, c='vrm', base='btc', result_only=True):
        return self.dispatch(
            requests.get(self.uri + '/public/getorderbook?market={}-{}&type=both'.format(base, c)),
            result_only)
    
    def orderbook_analysis(self, c='vrm', base='btc', sample=10):
        def get_score(arr):
            total_diff = 0
            for i in range(len(arr)-1):
                slope = abs(1/(arr[i+1] - arr[i]))
                total_diff += slope
            return total_diff
        
        orderbook = self.get_orderbook(c=c, base=base, result_only=True)
        
        total_buy_orders = 0
        buy_order_list   = []
        for buy_order in orderbook['buy'][:sample]: # {'Quantity/Size(ETH)': 2.26034925, 'Rate/Bid(BTC)': 0.04627} 
            quantity           = buy_order['Quantity']
            bid                = buy_order['Rate']
            buy_order_in_btc   = quantity * bid
            total_buy_orders   += buy_order_in_btc
            buy_order_list.append(total_buy_orders)
        
        total_sell_orders = 0
        sell_order_list   = []
        for sell_order in orderbook['sell'][:sample]: # {'Quantity/Size(ETH)': 2.26034925, 'Rate/Ask(BTC)': 0.04627} 
            quantity           = sell_order['Quantity']
            ask                = sell_order['Rate']
            sell_order_in_btc  = quantity * ask
            total_sell_orders  += sell_order_in_btc
            sell_order_list.append(total_sell_orders)
        
        return {'total_buy_orders': total_buy_orders,
                'total_sell_orders': total_sell_orders,
                'buy_orders': buy_order_list,
                'sell_orders': sell_order_list,
                'buy_order_slope': get_score(buy_order_list),
                'sell_order_slope': get_score(sell_order_list)}
    
    # all = True for all coins, all = ['ETH', 'VRM', ..] for specific coins
    # loop = 0 for a infinite loop, loop = 5 for 5 loops
    # sleep = 60 for a 60 second sleep time between two requests
    def write_serial(self, all=True, loop=0, sleep=60, folder=''):
        while True:
            try:
                coinlist = self.get_coins() if all == True else all
                loop = self.infinity() if loop == 0 else range(loop)

                for coin in coinlist:
                    coin     = coin.upper()
                    filepath = '{}{}.csv'.format(folder, coin)

                    if not os.path.isfile(filepath):

                        f = open(filepath, 'w')
                        f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(
                            "timestamp", "tick", "ask", "bid", "vol", "buys", 
                            "sells", "btc_tick", "btc_buys", "btc_sells", "btc_vol", "usdt",
                            "first_page_buy_orders", "first_page_sell_orders"))

                    for i in loop:
                        f = open(filepath, 'a')

                        coin_summary = self.get_summary(coin)[0]
                        timestamp    = coin_summary['TimeStamp']
                        tick         = coin_summary['Last']
                        ask          = coin_summary['Ask']
                        bid          = coin_summary['Bid']
                        vol          = coin_summary['BaseVolume']
                        buys         = coin_summary['OpenBuyOrders']
                        sells        = coin_summary['OpenSellOrders']

                        btc_market   = self.get_summary('btc', base='usdt')[0]
                        btc_tick     = btc_market['Last']
                        btc_buys     = btc_market['OpenBuyOrders']
                        btc_sells    = btc_market['OpenSellOrders']
                        btc_vol      = btc_market['Volume']

                        usdt         = tick * btc_tick
                        
                        try:
                            order_book   = self.orderbook_analysis(coin)
                            first_page_buy_orders = order_book['total_buy_orders']
                            first_page_sell_orders = order_book['total_sell_orders']
                        except:
                            first_page_buy_orders = 0
                            first_page_sell_orders = 0
                        

                        f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(
                            timestamp, tick, ask, bid, vol, buys, 
                            sells, btc_tick, btc_buys, btc_sells, btc_vol, 
                            usdt, first_page_buy_orders, first_page_sell_orders))

                        f.close()
                        time.sleep(sleep)
            except Exception as e:
                print(e)
                time.sleep(30)
    
    def write(self, all=True, loop=0, sleep=60, folder=''):
        coinlist = self.get_coins() if all == True else all        
        
        jobs = []
        for coin in coinlist:
            thread = multiprocessing.Process(target=self.write_serial, args=([coin], loop, sleep, folder))
            jobs.append(thread)
        
        for j in jobs:
            try:
                j.start()
            except:
                j.start()
            
        for j in jobs:
            j.join()

        
        
    def plot(self, c='vrm', base='btc', loop=0, sleep=10):
        try:
            from IPython.display import clear_output
            plt.rcParams['figure.figsize'] = (16,4)
        except:
            raise("This method only works on IPython notebook!")
            
        coin         = c.upper()
        loop         = self.infinity() if loop == 0 else range(loop)
        
        tick, ask, bid = [], [], []
        for i in loop:
            clear_output(wait=True)
            coin_summary = self.get_summary(coin, base=base)[0]
            
            tick.append(coin_summary['Last'])
            ask.append(coin_summary['Ask'])
            bid.append(coin_summary['Bid'])
            
            plt.plot(ask, color='r', label='ask')
            plt.plot(bid, color='g', label='bid')
            plt.plot(tick, color='b', label='tick')
            plt.legend()
            plt.show()
            
            time.sleep(sleep)
    def plot_file(self, c='vrm', path='data/', last=1000, density=1, outfile=False):
        if outfile:
            output_file("prediction.html", title="prediction")
        data = read_csv('{}{}.csv'.format(path,c.upper()), header=0, index_col=0)
        headers       = [i for i in data.axes[1]]
        data = data.values
        

        x = [i for i in range(data.shape[0])]
        
        

        plottick = figure(title='tick analysis', x_axis_label='time (1min per dot)', y_axis_label='value', 
                        width=950, height=300, sizing_mode='scale_width', 
                        tools = "xwheel_zoom,undo,redo,xpan,save,reset,hover,box_zoom",active_scroll="xwheel_zoom")
    
        x_tick = data[:,0].copy()
        x_ask  = data[:,1]
        x_sell = data[:,2]

        plottick.line(x, x_tick[-last::density], legend='tick', line_width=2, line_color='blue', alpha=0.8)
        plottick.line(x, x_ask[-last::density], legend='ask', line_width=1, line_color='red', alpha=0.8)
        plottick.line(x, x_sell[-last::density], legend='bid', line_width=1, line_color='green', alpha=0.8)

        
        ################################
        
        
        plotvol = figure(title='volume analysis', x_axis_label='time (1min per dot)', y_axis_label='value', 
                        width=950, height=300, sizing_mode='scale_width', 
                        tools = "xwheel_zoom,undo,redo,xpan,save,reset,hover,box_zoom",active_scroll="xwheel_zoom")

        minvol = data[:,4].min() if data[:,4].min() < data[:,5].min() else data[:,5].min()
        maxvol = data[:,4].max() if data[:,4].max() > data[:,5].max() else data[:,5].max()
    
    
        vol_tick = data[:,0].copy()
        vol_tick = self.scale(vol_tick, minvol, maxvol)
        
        plotvol.line(x, vol_tick[-last::density], legend='tick', line_width=2, line_color='blue', alpha=0.8)
        plotvol.line(x, data[:,4][-last::density], legend='buys', line_width=1, line_color='green', alpha=0.8)
        plotvol.line(x, data[:,5][-last::density], legend='sells', line_width=1, line_color='red', alpha=0.8)
        
        
        ################################
        
        
        plot1stpage = figure(title='1st page analysis', x_axis_label='time (1min per dot)', y_axis_label='value', 
                        width=950, height=300, sizing_mode='scale_width', 
                        tools = "xwheel_zoom,undo,redo,xpan,save,reset,hover,box_zoom",active_scroll="xwheel_zoom")
        
        minvol1stpage = data[:,11].min() if data[:,11].min() < data[:,12].min() else data[:,12].min()
        maxvol1stpage = data[:,11].max() if data[:,11].max() > data[:,12].max() else data[:,12].max()
        
        
        firstpage_tick = data[:,0].copy()
        firstpage_tick = self.scale(firstpage_tick, minvol1stpage, maxvol1stpage)
        
        plot1stpage.line(x, firstpage_tick[-last::density], legend='tick', line_width=2, line_color='blue', alpha=0.8)
        plot1stpage.line(x, data[:,11][-last::density], legend='1st page buys', line_width=1, line_color='green', alpha=0.8)
        plot1stpage.line(x, data[:,12][-last::density], legend='1st page sells', line_width=1, line_color='red', alpha=0.8)

        grid = gridplot([[plottick], [plotvol], [plot1stpage]])
        show(grid)
        
    def get(self, c='vrm', column='tick', path='data/', normalize=False):
        data = read_csv('{}{}.csv'.format(path,c.upper()), header=0)
        headers       = [i for i in data.axes[1]]
        data = data.values
        
        if column == 'timestamp'  : return data[:,0] if not normalize else self.scale(data[:,0], 0, 1)
        if column == 'tick'       : return data[:,1] if not normalize else self.scale(data[:,1], 0, 1)
        if column == 'ask'        : return data[:,2] if not normalize else self.scale(data[:,2], 0, 1)
        if column == 'bid'        : return data[:,3] if not normalize else self.scale(data[:,3], 0, 1)
        if column == 'vol'        : return data[:,4] if not normalize else self.scale(data[:,4], 0, 1)
        if column == 'buys'       : return data[:,5] if not normalize else self.scale(data[:,5], 0, 1)
        if column == 'sells'      : return data[:,6] if not normalize else self.scale(data[:,6], 0, 1)
        
        if column == 'btc_tick'   : return data[:,7] if not normalize else self.scale(data[:,7], 0, 1)
        if column == 'btc_buys'   : return data[:,8] if not normalize else self.scale(data[:,8], 0, 1)
        if column == 'btc_sells'  : return data[:,9] if not normalize else self.scale(data[:,9], 0, 1)
        if column == 'btc_vol'    : return data[:,10] if not normalize else self.scale(data[:,10], 0, 1)
        
        if column == 'usdt'       : return data[:,11] if not normalize else self.scale(data[:,11], 0, 1)
        if column == 'first_page_buy_orders'  : return data[:,12] if not normalize else self.scale(data[:,12], 0, 1)
        if column == 'first_page_sell_orders' : return data[:,13] if not normalize else self.scale(data[:,13], 0, 1)
    
#    def get_best_volume_coins(self, N=5):
        
    
    def plot_vol_dif(self, c='vrm', normalize=False, density=1, last=None):

        vol = self.get(c, 'vol')[::-1]
        tick = self.get(c, 'tick')[::-1]

        vol_diffs  = []
        tick_diffs = []
        for i in range(len(tick) - density):
            vol_val1  = math.sqrt( abs( vol[i]**2 - vol[i+density]**2 ))
            vol_val2  = vol[i] - vol[i+density]
            
            tick_val1 = math.sqrt( abs( tick[i]**2 - tick[i+density]**2 ))
            tick_val2 = tick[i] - tick[i+density]
            
            vol_diffs.append( vol_val2 )
            tick_diffs.append( tick_val2 )
            
        vol_diffs = self.scale(np.array(vol_diffs), 0, 1) if normalize else vol_diffs
        tick_diffs = self.scale(np.array(tick_diffs), 0, 1) if normalize else tick_diffs
        
        
        plt.plot(vol_diffs[:last], label=c+' - diffs')
        plt.plot(tick_diffs[:last], label=c+' - ticks')
        plt.legend()
        plt.show()
    
    
    def analysis(self, normalize=False, density=1):
        def get_diffs(data):
            diffs = []
            for i in range(1, len(data)):
                diff = data[i] - data[i-1]
                diffs.append(diff)
            return np.sum(diffs)
        
        all_buy_sell_diffs       = []
        first_page_buy_orders    = []
        first_page_sell_orders   = []
        
        volume_diffs             = []
        
        for coin in self.get_coins():
            sum_of_volumes = np.sum(self.get(coin, 'buys') - self.get(coin, 'sells'))
            all_buy_sell_diffs.append({ 'coin': coin, 'value': sum_of_volumes } )
            
            sum_of_buy_orders  = self.get(coin, 'first_page_buy_orders')[:1]
            sum_of_sell_orders = self.get(coin, 'first_page_sell_orders')[:1]
            first_page_buy_orders.append( { 'coin': coin, 'value': sum_of_buy_orders })
            first_page_sell_orders.append({ 'coin': coin, 'value': sum_of_sell_orders })
        
            volume_diffs.append({ 'coin': coin, 'value': get_diffs(self.get(coin, 'vol')) })
            
            
        
        
        all_buy_sell_diffs.sort(key=lambda x:x['value'], reverse=True)
        first_page_buy_orders.sort(key=lambda x:x['value'], reverse=True)
        first_page_sell_orders.sort(key=lambda x:x['value'], reverse=True)
        volume_diffs.sort(key=lambda x:x['value'], reverse=True)
        
        return {
                    'buy_sell_diffs'         : all_buy_sell_diffs, 
                    'first_page_buy_orders'  : first_page_buy_orders, 
                    'first_page_sell_orders' : first_page_sell_orders, 
                    'diffs'                  : volume_diffs
               }
    
    def plot_multi(self, coins=None, types=None, last=5000, density=1, normalize=True, plot='plt'):
        
        if plot == 'plt':
            for c in coins:
                for t in types:
                    plt.plot(self.get(c,column=t, normalize=normalize)[-last::density], label='{}_{}'.format(c,t))
            plt.legend()
            plt.show()
        else:
            colorlength = len(coins) + len(types)
            plotit = figure(title='analysis', x_axis_label='time', y_axis_label='value', 
                            width=950, height=300, sizing_mode='scale_width', 
                            tools = "xwheel_zoom,undo,redo,xpan,save,reset,hover,box_zoom",active_scroll="xwheel_zoom")

            x = [i for i in range(len(self.get(coins[0],column=types[0], normalize=normalize)[-last::density]))]
            colors = [color for m, color in zip(range(colorlength), palettes.Category20_20)]
            
            iters = 0
            for c in coins:
                for t in types:
                    plotit.line(x, self.get(c,column=t, normalize=normalize)[-last::density], 
                                 legend='{}_{}'.format(c,t), line_width=1, alpha=0.8, line_color=colors[iters%colorlength])
                    iters += 1

            show(plotit)
        

Loading BokehJS ...

In [128]:
a = Bittrex()

In [79]:
a.plot_file(c='vrm', last=1000, density=1)

/usr/local/lib/python3.6/dist-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 15829), ('y', 1000)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/dist-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 15829), ('y', 1000)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/dist-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 15829), ('y', 1000)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/dist-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the

In [ ]:
for i in a.get_coins():
    a.plot_multi(coins=[i], type='vol', last=1000, density=1, normalize=True)

In [ ]:
a.plot_multi(coins=['dgb','strat','ok','pink','snt'], type='first_page_sell_orders', last=10000, density=5, normalize=False)

In [ ]:
a.plot_multi(coins=['dgb'], type='first_page_sell_orders', last=1000, density=1, normalize=False)

In [132]:
a.plot_multi(coins=['dgb','strat','ok','pink','snt'], types=['tick'], last=500, density=1, normalize=True, plot='bokeh')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
a.orderbook_analysis('snt')['total_buy_orders']

In [99]:

palettes.Category20_20

['#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5',
 '#8c564b',
 '#c49c94',
 '#e377c2',
 '#f7b6d2',
 '#7f7f7f',
 '#c7c7c7',
 '#bcbd22',
 '#dbdb8d',
 '#17becf',
 '#9edae5']

In [111]:
[color for m, color in zip(range(10), palettes.Category20_20)]

['#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5']